In [1]:
import sympy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sympy import sin, cos, pi, Function
from sympy import Symbol, symbols, Matrix, Transpose, init_session, Array, tensorproduct
from sympy.physics.vector import ReferenceFrame, outer, dynamicsymbols, Point


## Definindo as funções para h e psi

In [2]:
#Defining h
def h(t, L, w, e_mais, e_cruzado,A):
	h_mais = A*cos(w*t-w*L)
	h_cruzado = A*sin(w*t-w*L)
	return h_mais*e_mais + h_cruzado*e_cruzado

\begin{equation}
 h = h_+ + h_\times
\end{equation}

In [3]:
#função PSI(t)
def PSIj(j, k, L, N, A, w, T, ep, ec):
    H = h(T, L[j-1], w, ep, ec,A) 
    phij = N[j-1].dot(H.dot(N[j-1]))/2
    return phij/(1-(k.dot(N[j-1]))**2) #expandir aqui

\begin{equation}
\Psi (t) = \frac{n^i h_{ij} n^j}{2(1 - (\hat{k}\cdot \hat{n})^2)}
\end{equation}

## Símbolos

In [4]:
phi, theta, t, w, L, A , psi, sigma= symbols('ϕ θ t ω L A ψ σ')

## Sistemas de coordenadas e vetores usando o sympy

In [5]:
DetFrame = ReferenceFrame("Det")
WaveFrame = ReferenceFrame("Wave")
WaveFrame.orient(DetFrame, "body", (phi, theta, psi), 'zxz')

In [6]:
vx = WaveFrame.x
vy = WaveFrame.y
vz = WaveFrame.z
dbaseii = outer(vx, vx)
dbaseij = outer(vx, vy)
dbaseik = outer(vx, vz)
dbaseji = outer(vy, vx)
dbasejj = outer(vy, vy)
dbasejk = outer(vy, vz)
dbaseki = outer(vz, vx)
dbasekj = outer(vz, vy)
dbasekk = outer(vz, vz)

In [7]:
e_plus  = dbaseii - dbasejj
e_cross = dbaseij + dbaseji

In [8]:
#n no referencial do detector
n2 = cos(sigma)*DetFrame.x + sin(sigma)*DetFrame.y
n3 = cos(sigma)*DetFrame.x - sin(sigma)*DetFrame.y

In [9]:
k = WaveFrame.z

## Defining posições dos satélites

In [10]:
O = Point('O') #origin

In [11]:
O.set_vel(DetFrame, 0)

In [12]:
#seting p1, p2, p3
p1 = Point(r'P_1') 
p2 = Point(r'P_2')
p3 = Point(r'P_3')
#r1, r2, r3, gamma1, gamma2, gamma3 = symbols(r'r_1 r_2 r_3 \gamma_1 \gamma_2 \gamma_3') #dist from org & phase angle
l = Symbol('l')
p1.set_pos(O, l*cos(0     )*DetFrame.x + l*sin(0     )*DetFrame.y + 0*DetFrame.z)
p2.set_pos(O, l*cos(2*pi/3)*DetFrame.x + l*sin(2*pi/3)*DetFrame.y + 0*DetFrame.z)
p3.set_pos(O, l*cos(4*pi/3)*DetFrame.x + l*sin(4*pi/3)*DetFrame.y + 0*DetFrame.z)

In [13]:
P1 = p1.pos_from(O)
P2 = p2.pos_from(O)
P3 = p3.pos_from(O)
P = [P1, P2, P3]

In [14]:
#setting n's, according to KTV notation
n1 = p2.pos_from(p3)
n2 = p3.pos_from(p1)
n3 = p1.pos_from(p2)
L1 = n1.magnitude()
L2 = n2.magnitude()
L3 = n3.magnitude()
N = [n1, n2, n3]
L = [L1, L2, L3]

## Início do cálculo do interferômetro

In [15]:
PARAMETERS = (k,L,N,P,A,w,t, e_plus, e_cross)

In [16]:
def delay(func, D):
    return func.subs(w*t, w*t - L[D-1])

def ygw(i,j,k,L,N,P,A,w,T, ep, ec):
    m = abs(6-i-j)-1
    return (1+ k.dot(N[m]))*\
           (PSIj(m, k, L, N, A, w, T + k.dot(P[i-1]) - L[m], ep, ec)\
            - PSIj(m,k, L, N, A, w, T + k.dot(P[j-1]), ep, ec))    #   # T + k.dot(P[i]) - L[m]) , T + k.dot(P[j]))

def ygwD(i,j,k,L,N,P,A,w,T, ep, ec, D): #Ygw com delay
    #delay = L[D]
    return delay(ygw(i,j,k,L,N,P,A,w,T, ep, ec), D)
    
def yij(i,j, parms = PARAMETERS):
    k,L,N,P,A,w,T, ep, ec = parms
    return ygw(i,j,k,L,N,P,A,w,T, ep, ec)

def yijD(i,j,D):
    return delay(yij(i,j),D)

def yijDD(i,j,D, E):
    return delay(delay(yij(i,j),D),E)

In [17]:
f = A*cos(w*t)
f

A*cos(t*ω)

In [18]:
delay(f, 2)

A*cos(t*ω - sqrt(3)*sqrt(l**2))

In [54]:
X =   (yij(3,1) + yijD(1,3,2))\
   + delay(delay((yij(2,1) + yijD(1,2,3)),2),2)\
   - (yij(2,1) + yijD(1,2,3))\
   - delay(delay(yij(3,1)+yijD(1,3,2),2),2)\
   - delay(delay(delay(delay(\
      (yij(3,1) + yijD(1,3,2))\
   + delay(delay((yij(2,1) + yijD(1,2,3)),2),2)\
   - (yij(2,1) + yijD(1,2,3))\
   - delay(delay(yij(3,1)+yijD(1,3,2),2),2)\
     ,2),2),3),3)
    

In [55]:
#X = sympy.trigsimp(X)

In [62]:
y1 = yijD(3,1,2) - yij(2,3)

In [63]:
#calculando M
X=sympy.trigsimp(y1)


KeyboardInterrupt: 

In [46]:
X=sympy.expand(X)

In [59]:
X

-(-(-sqrt(3)*l*((-sin(ψ)*sin(ϕ) + cos(θ)*cos(ψ)*cos(ϕ))*(A*(3*l*(-sin(ψ)*sin(ϕ)*cos(θ) + cos(ψ)*cos(ϕ))/2 - sqrt(3)*l*(sin(ψ)*cos(θ)*cos(ϕ) + sin(ϕ)*cos(ψ))/2)*(sin(ω*(-l*sin(θ)*sin(ϕ)/2 + sqrt(3)*l*sin(θ)*cos(ϕ)/2 + t))*cos(sqrt(3)*ω*sqrt(l**2)) - sin(sqrt(3)*ω*sqrt(l**2))*cos(ω*(-l*sin(θ)*sin(ϕ)/2 + sqrt(3)*l*sin(θ)*cos(ϕ)/2 + t))) - A*(sin(ω*(-l*sin(θ)*sin(ϕ)/2 + sqrt(3)*l*sin(θ)*cos(ϕ)/2 + t))*sin(sqrt(3)*ω*sqrt(l**2)) + cos(ω*(-l*sin(θ)*sin(ϕ)/2 + sqrt(3)*l*sin(θ)*cos(ϕ)/2 + t))*cos(sqrt(3)*ω*sqrt(l**2)))*(-sqrt(3)*l*(-sin(ψ)*sin(ϕ) + cos(θ)*cos(ψ)*cos(ϕ))/2 + 3*l*(-sin(ψ)*cos(ϕ) - sin(ϕ)*cos(θ)*cos(ψ))/2)) + (A*(3*l*(-sin(ψ)*sin(ϕ)*cos(θ) + cos(ψ)*cos(ϕ))/2 - sqrt(3)*l*(sin(ψ)*cos(θ)*cos(ϕ) + sin(ϕ)*cos(ψ))/2)*(sin(ω*(-l*sin(θ)*sin(ϕ)/2 + sqrt(3)*l*sin(θ)*cos(ϕ)/2 + t))*sin(sqrt(3)*ω*sqrt(l**2)) + cos(ω*(-l*sin(θ)*sin(ϕ)/2 + sqrt(3)*l*sin(θ)*cos(ϕ)/2 + t))*cos(sqrt(3)*ω*sqrt(l**2))) + A*(sin(ω*(-l*sin(θ)*sin(ϕ)/2 + sqrt(3)*l*sin(θ)*cos(ϕ)/2 + t))*cos(sqrt(3)*ω*sqrt(l**2)) - sin(s

In [ ]:
#M=sympy.trigsimp(M)

In [60]:
F_mais=X.coeff(cos(w*t))
F_cruzado=X.coeff(sin(w*t))

In [61]:
F_cruzado

0

In [ ]:
f_mais = sympy.lambdify([   phi, theta, w, A, l], F_mais)
f_cruzado = sympy.lambdify([phi, theta, w, A, l], F_cruzado)
M_eval = sympy.lambdify([phi, theta, w, A, l], M)

In [ ]:
f_mais

In [ ]:
#defining parameters
phi_value, theta_value = np.mgrid[-np.pi:np.pi:100j, 0:np.pi:100j]
arm=5e9/3e8 #segundos

f=10**-3 #Hz
freq=2*np.pi*f
a=1


In [ ]:
#atribuindos os valores acima nas funções
#                       [phi          , theta       , w   , A, r1, r2, r3, gamma1, gamma2, gamma3]
f_mais_data    = f_mais((phi_value), (theta_value), freq, a, arm)
f_cruzado_data = f_cruzado((phi_value),(theta_value), freq, a, arm)


In [ ]:
f_mais_data

In [ ]:
#plot phi, theta e F
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(phi_value, theta_value,(f_mais_data),color='b')
#ax.plot_surface(phi_value, theta_value,(f_cruzado_data),color='g')
#ax.plot_surface(phi_value, theta_value,(f_cruzado_data-f_mais_data),color='g')
ax.set_xlabel('phi')
ax.set_ylabel('theta')
ax.set_zlabel('F+')
plt.show()



In [ ]:
#plot x,y,z

fig = plt.figure()
ax = fig.gca(projection='3d')

x_mais=(f_mais_data)*np.sin(theta_value)*np.sin(phi_value)
y_mais=-(f_mais_data)*np.sin(theta_value)*np.cos(phi_value)
z_mais=(f_mais_data)*np.cos(theta_value)

x_cruzado=(f_cruzado_data)*np.sin(theta_value)*np.sin(phi_value)
y_cruzado=-(f_cruzado_data)*np.sin(theta_value)*np.cos(phi_value)
z_cruzado=(f_cruzado_data)*np.cos(theta_value)

ax.plot_surface(x_mais,y_mais,z_mais,color='b')
#ax.plot_surface(x_cruzado,y_cruzado,z_cruzado,color='g')
#ax.plot_surface((x_cruzado-x_mais),(y_cruzado-y_mais),(z_cruzado-z_mais),color='g', label = 'F_cruzado')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()